In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Impor the OpenWeatherMap API key
from api_keys import weather_api_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

In [4]:
# Empty list for holding the latitude and longitude combinations
lat_lngs = []

# Empty list for holding the cities names
cities = []

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(f"Number of cities in the list: {len(cities)}")

Number of cities in the list: 585


In [5]:
# base Url
url = "https://api.openweathermap.org/data/2.5/weather"
## Define an empty list to fetch the weather data for each city
city_data = []

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")
# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0
        
    city_url = f"{url}?q={city}&appid={weather_api_key}"

    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
         # Parse the JSON and retrieve data
        city_weather = city_weather = requests.get(city_url).json()

        # Parse out latitude, longitude, max temp, humidity, cloudiness, wind speed, country, and date
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_date = city_weather["dt"]

        # Append the City information into city_data list
        city_data.append({"City": city, 
                          "Lat": city_lat, 
                          "Lng": city_lng, 
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
              
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | kruisfontein
Processing Record 2 of Set 1 | klyuchi
Processing Record 3 of Set 1 | bria
Processing Record 4 of Set 1 | waitangi
Processing Record 5 of Set 1 | beau vallon
Processing Record 6 of Set 1 | walvis bay
Processing Record 7 of Set 1 | college
Processing Record 8 of Set 1 | haiku-pauwela
Processing Record 9 of Set 1 | olyokminsk
Processing Record 10 of Set 1 | ribeira grande
Processing Record 11 of Set 1 | greymouth
Processing Record 12 of Set 1 | san andros
City not found. Skipping...
Processing Record 13 of Set 1 | tiksi
Processing Record 14 of Set 1 | margaret river
Processing Record 15 of Set 1 | adamstown
Processing Record 16 of Set 1 | enewetak
Processing Record 17 of Set 1 | talnakh
Processing Record 18 of Set 1 | kodiak
Processing Record 19 of Set 1 | blackmans bay
Processing Record 20 of Set 1 | fale old settlement
Processing Record 21 of Set 1 | grytviken
Processing Record 22 of

In [6]:
city_data_df = pd.DataFrame(city_data)

# Show the record count
print(city_data_df.count())

City          557
Lat           557
Lng           557
Max Temp      557
Humidity      557
Cloudiness    557
Wind Speed    557
Country       557
Date          557
dtype: int64


In [7]:
# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,kruisfontein,-34.0033,24.7314,291.42,85,100,4.45,ZA,1712538773
1,klyuchi,52.2667,79.1667,276.37,86,100,1.31,RU,1712538667
2,bria,6.5423,21.9863,293.61,89,100,1.59,CF,1712538774
3,waitangi,-43.9535,-176.5597,286.94,81,100,1.34,NZ,1712538658
4,beau vallon,-20.4189,57.6953,298.27,94,20,2.57,MU,1712538775


In [9]:
# Export the City_Data into a csv
city_data_df.to_csv("output_data/cities.csv", index_label="City_ID")